Generate, load and prepare policy histories

In [1]:
import os
from datetime import datetime
import support_functions as sf


PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

In [2]:
#sf.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#sf.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    sf.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    sf.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)

In [3]:
# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     sf.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)

# policy_histories_test = \
#     sf.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)

# Save padded data, since always generating and padding takes too long
#policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
#policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

Prepare labels and features

In [4]:
# Extract features and labels from dataset as numpy.ndarray(s)
train_labels, train_features, train_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length)
test_labels, test_features, test_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length)


Build TensorFlow graph with RNN

In [5]:
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(42)

# Our hyperparameters
n_steps = max_policy_history_length
n_inputs = 2
n_layers = 3
n_neurons = 100
n_outputs = 2
learning_rate = 0.0001

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None], name="seq_length")
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                      activation=tf.nn.relu)
          for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers, state_is_tuple=False)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# We feed all layers' states (after the last timestep) into a fully connected layer of 2 neurons
# (1 per class, troubled or not). Softmax layer is next
logits = tf.layers.dense(states, n_outputs)
y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

# Loss function and Adam Optimizer
loss = tf.reduce_mean(tf.cast(xentropy, tf.float32))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

accuracy_summary = tf.summary.scalar('Accuracy', accuracy)
loss_summary = tf.summary.scalar('Loss', loss)

file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

n_epochs = 100
batch_size = 200

train_data = sf.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(train_data.num_examples // batch_size):
            y_batch, X_batch, seq_length_batch = train_data.next_batch(batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})

        y_train_pred = y_pred.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        y_test_pred = y_pred.eval(feed_dict={X: test_features, y: test_labels, seq_length: test_seq_lengths})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        acc_test = accuracy.eval(feed_dict={X: test_features, y: test_labels, seq_length: test_seq_lengths})

        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        print(confusion_matrix(y_batch, y_train_pred))
        print(classification_report(y_batch, y_train_pred, target_names=['ok', 'troubled']))

        loss_summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        accuracy_summary_str = accuracy_summary.eval(feed_dict={X: test_features, y: test_labels, seq_length: test_seq_lengths})
        file_writer.add_summary(loss_summary_str, epoch)
        file_writer.add_summary(accuracy_summary_str, epoch)

file_writer.close()

0 Train accuracy: 0.45 Test accuracy: 0.4445
[[51 92]
 [18 39]]
             precision    recall  f1-score   support

         ok       0.74      0.36      0.48       143
   troubled       0.30      0.68      0.41        57

avg / total       0.61      0.45      0.46       200



1 Train accuracy: 0.95 Test accuracy: 0.951
[[146   1]
 [  9  44]]
             precision    recall  f1-score   support

         ok       0.94      0.99      0.97       147
   troubled       0.98      0.83      0.90        53

avg / total       0.95      0.95      0.95       200



2 Train accuracy: 0.93 Test accuracy: 0.933
[[149   0]
 [ 14  37]]
             precision    recall  f1-score   support

         ok       0.91      1.00      0.96       149
   troubled       1.00      0.73      0.84        51

avg / total       0.94      0.93      0.93       200



3 Train accuracy: 0.96 Test accuracy: 0.9685
[[153   0]
 [  8  39]]
             precision    recall  f1-score   support

         ok       0.95      1.00      0.97       153
   troubled       1.00      0.83      0.91        47

avg / total       0.96      0.96      0.96       200



4 Train accuracy: 0.945 Test accuracy: 0.926
[[154   3]
 [  8  35]]
             precision    recall  f1-score   support

         ok       0.95      0.98      0.97       157
   troubled       0.92      0.81      0.86        43

avg / total       0.94      0.94      0.94       200



...

95 Train accuracy: 1.0 Test accuracy: 0.9985
[[155   0]
 [  0  45]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       155
   troubled       1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00       200



96 Train accuracy: 1.0 Test accuracy: 0.9995
[[151   0]
 [  0  49]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       151
   troubled       1.00      1.00      1.00        49

avg / total       1.00      1.00      1.00       200



97 Train accuracy: 1.0 Test accuracy: 0.9995
[[149   0]
 [  0  51]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       149
   troubled       1.00      1.00      1.00        51

avg / total       1.00      1.00      1.00       200



98 Train accuracy: 1.0 Test accuracy: 0.999
[[162   0]
 [  0  38]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       162
   troubled       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00       200



99 Train accuracy: 1.0 Test accuracy: 0.999
[[151   0]
 [  0  49]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       151
   troubled       1.00      1.00      1.00        49

avg / total       1.00      1.00      1.00       200

